In [6]:
# Import all the required libraries
import fastf1
import pandas as pd

# Enable caching for faster data loading
fastf1.Cache.enable_cache('cache') 

In [8]:
# Import the results_df dataset
results_df = pd.read_csv('model/DATA/f1_results_2024_2025.csv')

In [39]:
# Load driver data from an Excel file
drivers = pd.read_excel('model/DATA/f1_drivers_points_exp.xlsx')
drivers

,DriverName,Abbreviation,TeamId,DriverExperience
0,Max Verstappen,VER,red_bull,11
1,Lewis Hamilton,HAM,ferrari,19
2,Charles Leclerc,LEC,ferrari,8
3,Carlos Sainz,SAI,williams,11
4,Lando Norris,NOR,mclaren,7
5,George Russell,RUS,mercedes,7
6,Fernando Alonso,ALO,aston_martin,22
7,Oscar Piastri,PIA,mclaren,3
8,Esteban Ocon,OCO,haas,9
9,Pierre Gasly,GAS,alpine,9


In [11]:
filtered_results = results_df[results_df["Abbreviation"].isin(drivers['Abbreviation'])]

# Group by driver and sum total points
total_points = filtered_results.groupby("Abbreviation")["Points"].sum().reset_index()

# Fill the missing rookie drivers with 0 points
all_drivers_df = pd.DataFrame({"Abbreviation": drivers["Abbreviation"]}) 
total_points = all_drivers_df.merge(total_points, on="Abbreviation", how="left").fillna(0) 

# Convert Points to absolute value interger for simpler point system
total_points["Points"] = total_points["Points"].astype(int)
total_points = total_points.sort_values(by="Points", ascending=False)

total_points

,Abbreviation,Points
0,VER,429
4,NOR,387
2,LEC,331
7,PIA,292
3,SAI,263
5,RUS,256
1,HAM,208
6,ALO,70
13,HUL,42
9,GAS,40


In [12]:
drivers = total_points.merge(drivers, on="Abbreviation", how="left").fillna(0) 
drivers

,Abbreviation,Points,DriverName,TeamId,DriverExperience
0,VER,429,Max Verstappen,red_bull,11
1,NOR,387,Lando Norris,mclaren,7
2,LEC,331,Charles Leclerc,ferrari,8
3,PIA,292,Oscar Piastri,mclaren,3
4,SAI,263,Carlos Sainz,williams,11
5,RUS,256,George Russell,mercedes,7
6,HAM,208,Lewis Hamilton,ferrari,19
7,ALO,70,Fernando Alonso,aston_martin,22
8,HUL,42,Nico Hülkenberg,sauber,13
9,GAS,40,Pierre Gasly,alpine,9


In [13]:
# Calculate average qualifying position for each driver
avg_quali_df = results_df.groupby('Abbreviation')['GridPosition'].mean().reset_index()
avg_quali_df.rename(columns={'GridPosition': 'AvgQualiPosition'}, inplace=True)

In [18]:
# Calculate average race position for each driver
avg_race_df = results_df.groupby('Abbreviation')['Position'].mean().reset_index()
avg_race_df.rename(columns={'Position': 'AvgRacePosition'}, inplace=True)

In [20]:
# Merge both dataframes
avg_pos_df = avg_quali_df.merge(avg_race_df, on='Abbreviation', how='outer')

# Merge the drivers DataFrame with avg_pos_df to include AvgQualiPosition and AvgRacePosition
drivers = drivers.merge(avg_pos_df, on='Abbreviation', how='left')

# Display updated drivers dataset
drivers

,Abbreviation,Points,DriverName,TeamId,DriverExperience,AvgQualiPosition,AvgRacePosition
0,VER,429,Max Verstappen,red_bull,11,3.538462,3.576923
1,NOR,387,Lando Norris,mclaren,7,3.269231,4.076923
2,LEC,331,Charles Leclerc,ferrari,8,5.500000,5.192308
3,PIA,292,Oscar Piastri,mclaren,3,5.115385,5.115385
4,SAI,263,Carlos Sainz,williams,11,6.240000,6.360000
5,RUS,256,George Russell,mercedes,7,5.423077,6.461538
6,HAM,208,Lewis Hamilton,ferrari,19,8.653846,7.538462
7,ALO,70,Fernando Alonso,aston_martin,22,9.653846,10.692308
8,HUL,42,Nico Hülkenberg,sauber,13,12.076923,11.576923
9,GAS,40,Pierre Gasly,alpine,9,13.076923,13.076923


In [22]:
# Remove the 'TeamId' column from the drivers DataFrame as it's not needed
filtered_drivers_info = drivers.drop(['DriverExperience'], axis=1)
filtered_drivers_info

,Abbreviation,Points,DriverName,TeamId,AvgQualiPosition,AvgRacePosition
0,VER,429,Max Verstappen,red_bull,3.538462,3.576923
1,NOR,387,Lando Norris,mclaren,3.269231,4.076923
2,LEC,331,Charles Leclerc,ferrari,5.500000,5.192308
3,PIA,292,Oscar Piastri,mclaren,5.115385,5.115385
4,SAI,263,Carlos Sainz,williams,6.240000,6.360000
5,RUS,256,George Russell,mercedes,5.423077,6.461538
6,HAM,208,Lewis Hamilton,ferrari,8.653846,7.538462
7,ALO,70,Fernando Alonso,aston_martin,9.653846,10.692308
8,HUL,42,Nico Hülkenberg,sauber,12.076923,11.576923
9,GAS,40,Pierre Gasly,alpine,13.076923,13.076923


In [24]:
filtered_drivers_info.to_csv('model/DATA/filtered_drivers_info.csv')

In [26]:
# Select only relevant features from results_df for further processing
filtered_results_df = results_df[['Round', 'Abbreviation', 'GridPosition', 'Position']]
filtered_results_df

,Round,Abbreviation,GridPosition,Position
0,1,VER,1.0,1.0
1,1,SAI,4.0,3.0
2,1,LEC,2.0,4.0
3,1,RUS,3.0,5.0
4,1,NOR,7.0,6.0
...,...,...,...,...
379,2,TSU,9.0,16.0
380,2,ALO,13.0,17.0
381,2,LEC,6.0,18.0
382,2,HAM,5.0,19.0


In [28]:
# Merge the filtered race results with driver information using 'Abbreviation' as the key
# This combines race performance with driver statistics and gives the final dataset to use for prediction
final_df = filtered_results_df.merge(filtered_drivers_info, on='Abbreviation', how='left')
final_df

,Round,Abbreviation,GridPosition,Position,Points,DriverName,TeamId,AvgQualiPosition,AvgRacePosition
0,1,VER,1.0,1.0,429,Max Verstappen,red_bull,3.538462,3.576923
1,1,SAI,4.0,3.0,263,Carlos Sainz,williams,6.240000,6.360000
2,1,LEC,2.0,4.0,331,Charles Leclerc,ferrari,5.500000,5.192308
3,1,RUS,3.0,5.0,256,George Russell,mercedes,5.423077,6.461538
4,1,NOR,7.0,6.0,387,Lando Norris,mclaren,3.269231,4.076923
...,...,...,...,...,...,...,...,...,...
379,2,TSU,9.0,16.0,29,Yuki Tsunoda,rb,10.769231,13.230769
380,2,ALO,13.0,17.0,70,Fernando Alonso,aston_martin,9.653846,10.692308
381,2,LEC,6.0,18.0,331,Charles Leclerc,ferrari,5.500000,5.192308
382,2,HAM,5.0,19.0,208,Lewis Hamilton,ferrari,8.653846,7.538462


In [30]:
final_df["QualifyingScore"] = (final_df["AvgQualiPosition"] + final_df["GridPosition"]) / 2
final_df

,Round,Abbreviation,GridPosition,Position,Points,DriverName,TeamId,AvgQualiPosition,AvgRacePosition,QualifyingScore
0,1,VER,1.0,1.0,429,Max Verstappen,red_bull,3.538462,3.576923,2.269231
1,1,SAI,4.0,3.0,263,Carlos Sainz,williams,6.240000,6.360000,5.120000
2,1,LEC,2.0,4.0,331,Charles Leclerc,ferrari,5.500000,5.192308,3.750000
3,1,RUS,3.0,5.0,256,George Russell,mercedes,5.423077,6.461538,4.211538
4,1,NOR,7.0,6.0,387,Lando Norris,mclaren,3.269231,4.076923,5.134615
...,...,...,...,...,...,...,...,...,...,...
379,2,TSU,9.0,16.0,29,Yuki Tsunoda,rb,10.769231,13.230769,9.884615
380,2,ALO,13.0,17.0,70,Fernando Alonso,aston_martin,9.653846,10.692308,11.326923
381,2,LEC,6.0,18.0,331,Charles Leclerc,ferrari,5.500000,5.192308,5.750000
382,2,HAM,5.0,19.0,208,Lewis Hamilton,ferrari,8.653846,7.538462,6.826923


In [32]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Round             384 non-null    int64  
 1   Abbreviation      384 non-null    object 
 2   GridPosition      384 non-null    float64
 3   Position          384 non-null    float64
 4   Points            384 non-null    int32  
 5   DriverName        384 non-null    object 
 6   TeamId            384 non-null    object 
 7   AvgQualiPosition  384 non-null    float64
 8   AvgRacePosition   384 non-null    float64
 9   QualifyingScore   384 non-null    float64
dtypes: float64(5), int32(1), int64(1), object(3)
memory usage: 28.6+ KB


In [34]:
# Count and find if there are any null values in the dataset
final_df.isnull().sum()

Round               0
Abbreviation        0
GridPosition        0
Position            0
Points              0
DriverName          0
TeamId              0
AvgQualiPosition    0
AvgRacePosition     0
QualifyingScore     0
dtype: int64

In [36]:
final_df.to_csv('model/DATA/f1_final_data.csv', index=False)